In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import csv
import json

# Import API key
from config import g_key

In [2]:
#Load the csv exported in Part I to a DataFrame

cities_df = pd.read_csv("cities.csv")
cities_df.head()

,Unnamed: 0,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Kalmunai,7.42,81.82,79.61,76,97,2.46,LK,1603662340
1,1,Torbay,47.67,-52.73,41.27,80,75,20.80,CA,1603662340
2,2,Antsohihy,-14.88,47.99,75.85,58,27,5.99,MG,1603662340
3,3,Ixtapa,20.70,-105.20,87.91,74,20,9.17,MX,1603662340
4,4,Codrington,-38.27,141.97,51.96,70,0,18.77,AU,1603662340


In [3]:
# Humidity Heatmap
# Configure gmaps

gmaps.configure(api_key=g_key)

In [4]:
# Use the Lat and Lng as locations and Humidity as the weight.
# Add Heatmap layer to map.

# Store 'Lat' and 'Lng' into  locations 
locations = cities_df[["Latitude", "Longitude"]]
locations.head

rating = cities_df["Humidity"]

fig=gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights = rating, dissipating=False, max_intensity=10, point_radius=1)
fig.add_layer(heat_layer)
fig


Figure(layout=FigureLayout(height='420px'))

In [5]:
# Create new DataFrame fitting weather criteria
# Narrow down the cities to fit weather conditions.
cities_df
ideal_city_data_df = cities_df.loc[cities_df["Wind Speed"]<6,:]
ideal_city_data2_df = ideal_city_data_df.loc[ideal_city_data_df["Max Temp"]<85,:]
ideal_city_data3_df = ideal_city_data2_df.loc[ideal_city_data2_df["Humidity"]<35,:]
# Drop any rows will null values.
hotel_df = ideal_city_data3_df.dropna(how='any')
hotel_df

,Unnamed: 0,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
19,19,Faya,18.39,42.45,62.60,17,0,1.99,SA,1603662324
33,33,Bilma,18.69,12.92,72.00,22,0,3.47,NE,1603662256
72,72,Tikrit,34.61,43.68,64.33,26,2,3.65,IQ,1603662352
90,90,Zambezi,-13.54,23.10,71.47,26,11,2.98,ZM,1603662355
242,242,Sarakhs,36.54,61.16,50.00,34,0,4.70,IR,1603662386
306,306,Viedma,-40.81,-63.00,66.99,27,0,4.00,AR,1603662396
331,331,Menongue,-14.66,17.69,68.79,25,0,3.06,AO,1603662400
371,371,Sakakah,29.97,40.21,69.80,12,20,5.82,SA,1603662407
421,421,Viranşehir,37.24,39.76,65.32,26,60,3.31,TR,1603662414
509,509,Abhā,18.22,42.51,62.47,17,0,1.59,SA,1603662428


In [6]:
#add columns to dataframe
choicehotels_df = hotel_df
choicehotels_df["Hotel Name"]=""
choicehotels_df["Hotel Lat"]=""
choicehotels_df["Hotel Lng"]=""
#print dataframe
choicehotels_df

,Unnamed: 0,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name,Hotel Lat,Hotel Lng
19,19,Faya,18.39,42.45,62.60,17,0,1.99,SA,1603662324,,,
33,33,Bilma,18.69,12.92,72.00,22,0,3.47,NE,1603662256,,,
72,72,Tikrit,34.61,43.68,64.33,26,2,3.65,IQ,1603662352,,,
90,90,Zambezi,-13.54,23.10,71.47,26,11,2.98,ZM,1603662355,,,
242,242,Sarakhs,36.54,61.16,50.00,34,0,4.70,IR,1603662386,,,
306,306,Viedma,-40.81,-63.00,66.99,27,0,4.00,AR,1603662396,,,
331,331,Menongue,-14.66,17.69,68.79,25,0,3.06,AO,1603662400,,,
371,371,Sakakah,29.97,40.21,69.80,12,20,5.82,SA,1603662407,,,
421,421,Viranşehir,37.24,39.76,65.32,26,60,3.31,TR,1603662414,,,
509,509,Abhā,18.22,42.51,62.47,17,0,1.59,SA,1603662428,,,


In [7]:
#define url
hotel_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
for index, row in choicehotels_df.iterrows():
#define parameters    
    params ={
        "location": "{} ,{} ".format(row["Latitude"],row["Longitude"]),
        "rankby": "distance",
        "type": "lodging",
        "language": "en",
        "key": g_key
    }    
    response = requests.get(hotel_url, params=params).json()
#add new values to dataframe    
    choicehotels_df.loc[index, "Hotel Name"]=response["results"][0]["name"]
    choicehotels_df.loc[index, "Hotel Lat"]=response["results"][0]["geometry"]["location"]["lat"]
    choicehotels_df.loc[index, "Hotel Lng"]=response["results"][0]["geometry"]["location"]["lng"]    
    
#print dataframe    
choicehotels_df   

,Unnamed: 0,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name,Hotel Lat,Hotel Lng
19,19,Faya,18.39,42.45,62.60,17,0,1.99,SA,1603662324,مقبرة ال مجمل,18.3895,42.4549
33,33,Bilma,18.69,12.92,72.00,22,0,3.47,NE,1603662256,Expeditions Ténére voyages,18.693,12.9177
72,72,Tikrit,34.61,43.68,64.33,26,2,3.65,IQ,1603662352,فندق تكريت بلازا,34.6052,43.6714
90,90,Zambezi,-13.54,23.10,71.47,26,11,2.98,ZM,1603662355,Zambezi Motel,-13.54,23.1004
242,242,Sarakhs,36.54,61.16,50.00,34,0,4.70,IR,1603662386,Sarakhs Abrisham Hotel,36.5362,61.1648
306,306,Viedma,-40.81,-63.00,66.99,27,0,4.00,AR,1603662396,Alta Patagonia Apart,-40.8091,-63.0006
331,331,Menongue,-14.66,17.69,68.79,25,0,3.06,AO,1603662400,Complexo Turístico Reencontro,-14.664,17.6893
371,371,Sakakah,29.97,40.21,69.80,12,20,5.82,SA,1603662407,ناصر عزام الشايع,29.9673,40.2103
421,421,Viranşehir,37.24,39.76,65.32,26,60,3.31,TR,1603662414,Çetin Kardeşler Oto Yıkama,37.232,39.7542
509,509,Abhā,18.22,42.51,62.47,17,0,1.59,SA,1603662428,شقق مفروشة,18.2202,42.5106


In [8]:
# Add marker layer ontop of heat map

marker_locations = []
for index, row in choicehotels_df.iterrows():
    marker_locations.append(tuple([row["Hotel Lat"],row["Hotel Lng"]]))
markers = gmaps.marker_layer(marker_locations)

fig.add_layer(markers)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))